In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold

In [2]:
import os

diretorio_base = "./data/espectogramas/"

# Lista de arrays no formato [[key, nome], ...]
lista_arquivos = []

for pasta in os.listdir(diretorio_base):
    if os.path.isdir(os.path.join(diretorio_base, pasta)):
        diretorio_pasta = os.path.join(diretorio_base, pasta)
        for arquivo in os.listdir(diretorio_pasta):
            lista_arquivos.append([pasta, arquivo])


df_X = pd.DataFrame(lista_arquivos, columns=['key', 'nome_arquivo'])


# Remova o elemento em que só há uma ocorrência de determinada 'key'
df_X = df_X.groupby('key').filter(lambda x: len(x) > 1)

# transforma em lista
lista_arquivos = df_X.values.tolist()

# Separa a primeira coluna da lista na variavel nome_arquivo e key
key, nome_arquivo = zip(*lista_arquivos)

# Remove o .png do nome do arquivo
nome_arquivo = [nome[:-4] for nome in nome_arquivo]





In [3]:
arq_treino, arq_teste, y_treino, y_teste = train_test_split(nome_arquivo, key, test_size=0.2, stratify=key)


In [19]:
# Carregar o arquivo LBP.csv
df_lbp = pd.read_csv('./data/caracteristicas/LBP.csv')

# Filtrar as amostras com 'nome_arquivo' presente em arq_treino
df_train = df_lbp[df_lbp['nome_arquivo'].isin(arq_treino)]

# Filtrar as amostras com 'nome_arquivo' presente em arq_teste
df_test = df_lbp[df_lbp['nome_arquivo'].isin(arq_teste)]

# Salvar os arquivos
df_train.to_csv('./data/caracteristicas/LBP_treino.csv', index=False)
df_test.to_csv('./data/caracteristicas/LBP_teste.csv', index=False)

# iporta o knn
from sklearn.neighbors import KNeighborsClassifier

# Carrega o arquivo de treino
df_train = pd.read_csv('./data/caracteristicas/LBP_treino.csv')

# Carrega o arquivo de teste
df_test = pd.read_csv('./data/caracteristicas/LBP_teste.csv')

# Separa as colunas de entrada e saida
y_train = df_train['key']
X_train = df_train.drop(columns=['key', 'nome_arquivo'])

y_test = df_test['key']
X_test = df_test.drop(columns=['key', 'nome_arquivo'])

# normaliza os dados
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Ajusta os dados de treino
X_train = scaler.fit_transform(X_train)

# Ajusta os dados de teste
X_test = scaler.transform(X_test)

# Cria o classificador
knn = KNeighborsClassifier(n_neighbors=11)

# Treina o classificador
knn.fit(X_train, y_train)

# Testa o classificador
y_pred = knn.predict(X_test)

# Calcula a acurácia
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred)
print('Acurácia: ', acc)


Acurácia:  0.07751285233114696
